In [1]:
import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

In [2]:
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/kogpt2_news_wiki_ko_cased_818bfa919d.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')
# model = TFGPT2LMHeadModel.from_pretrained('./gpt_ckpt')

In [3]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [4]:
gpt_model = GPT2Model('./gpt_ckpt')

In [5]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    gen_count = 100

    toked = tokenizer(sent)
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [6]:
generate_sent('이때', gpt_model, max_step=10, greedy=True)

'이때문에 일부 전문가들은 “이번 사건은 ‘제2의'

In [7]:
generate_sent('이때', gpt_model, top_k=30, top_p=0.95)

'이때 \'귀태\'의 저자 최인호는 "막말이 정치인에 대한 인신공격으로 이어지지는 않을 것"이라면서도 "다만 박근혜는 (박근혜 대통령의) 국정철학에 대해 많은 것을 알게 될 것"이라고 말했다.'

In [8]:
sents = [s[:-1] for s in open('finetune_data.txt').readlines()]
input_data = list()
output_data = list()

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [10]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [11]:
# overfitting을 막기 위한 ealrystop 추가
# earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=1)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

history = gpt_model.fit(input_data, output_data, 
                    batch_size=16, epochs=10,
                    validation_split=0.1)#, callbacks=[earlystop_callback])


# history = model.fit(dataset, epochs=2)#, callbacks=[earlystop_callback])

Train on 255 samples, validate on 29 samples
Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


255/255 [==============================] - 69s 271ms/sample - loss: 2.9908 - accuracy_function: 0.1000 - val_loss: 2.4743 - val_accuracy_function: 0.1148
Epoch 2/10
255/255 [==============================] - 48s 189ms/sample - loss: 2.5006 - accuracy_function: 0.1256 - val_loss: 2.3912 - val_accuracy_function: 0.1326
Epoch 3/10
255/255 [==============================] - 47s 186ms/sample - loss: 2.2519 - accuracy_function: 0.1412 - val_loss: 2.3786 - val_accuracy_function: 0.1472
Epoch 4/10
255/255 [==============================] - 48s 186ms/sample - loss: 2.0414 - accuracy_function: 0.1549 - val_loss: 2.3757 - val_accuracy_function: 0.1597
Epoch 5/10
255/255 [==============================] - 48s 188ms/sample - loss: 1.8267 - accuracy_function: 0.1676 - val_loss: 2.4177 - val_accuracy_function: 0.1724
Epoch 6/10
255/255 [==============================] - 49s 193ms/sample - loss: 1.6278 - accuracy_function: 0.1787 - val_loss: 2.4846 - val_accuracy_function: 0.1841
Epoch 7/10
255/255 [=

In [16]:
generate_sent('이때', gpt_model, greedy=True)

'이때에 마침 길가 선술집에서 김첨지는 길가 선술집에 있는 이의 다급한 목소리와는 달리 매우 반가운 듯이 손<unk>하였다.'

In [ ]:
generate_sent('이때', gpt_model, top_k=30, top_p=0.95)